In [1]:
from floodligt_functions import clean_metadata
from floodligt_functions import read_position_data_jsonl

import numpy as np
from pathlib import Path
import json
import io
import polars as pl
import floodlight
#from floodlight.io.secondspectrum import read_position_data_jsonl
from floodlight.io.secondspectrum import read_teamsheets_from_meta_json
import json
import tempfile
from typing import Union, Tuple, Dict
from pathlib import Path
from floodlight.core.pitch import Pitch
from floodlight.io.utils import get_and_convert
from floodlight.core.teamsheet import Teamsheet
from floodlight.core.xy import XY
from floodlight.core.code import Code
from floodlight.models.kinematics import AccelerationModel
from floodlight.models.kinematics import DistanceModel
from floodlight.models.kinematics import VelocityModel
from floodlight.core.property import PlayerProperty
from floodlight.models.base import BaseModel, requires_fit
from floodlight.models.kinematics import VelocityModel, AccelerationModel
from floodlight.models.kinetics import MetabolicPowerModel


In [2]:
velocity=np.load("np_arrays/velocity.npy")

In [3]:
acceleration=np.load("np_arrays/acceleration.npy")

In [4]:
xy=np.load("np_arrays/xy.npy")

In [5]:
xy

array([[39.94, -0.09,   nan, ...,   nan,   nan,   nan],
       [39.8 , -0.09,   nan, ...,   nan,   nan,   nan],
       [39.67, -0.09,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan]],
      shape=(2901801, 42))

In [6]:
equivalent_slope = MetabolicPowerModel._calc_es(
            velocity, acceleration
        )

In [7]:
equivalent_mass = MetabolicPowerModel._calc_em(equivalent_slope)

In [8]:
xy

array([[39.94, -0.09,   nan, ...,   nan,   nan,   nan],
       [39.8 , -0.09,   nan, ...,   nan,   nan,   nan],
       [39.67, -0.09,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan]],
      shape=(2901801, 42))

In [9]:
metabolic_power = MetabolicPowerModel._calc_metabolic_power(
            equivalent_slope, velocity, equivalent_mass, 25, 3.6
        )

: 

In [ ]:
equivalent_mass